In [2]:
import pandas as pd

/var/folders/9k/3f86r_td4ygg_zflr8hnwkjr0000gn/T/ipykernel_32264/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
data = pd.read_csv('../datasets/car_data.csv')
data.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [8]:
print(data['Car_Name'].unique())
print(data.shape[0])
print(data['Seller_Type'].unique())
print(data['Transmission'].unique())
print(data['Fuel_Type'].unique())

['ritz' 'sx4' 'ciaz' 'wagon r' 'swift' 'vitara brezza' 's cross'
 'alto 800' 'ertiga' 'dzire' 'alto k10' 'ignis' '800' 'baleno' 'omni'
 'fortuner' 'innova' 'corolla altis' 'etios cross' 'etios g' 'etios liva'
 'corolla' 'etios gd' 'camry' 'land cruiser' 'Royal Enfield Thunder 500'
 'UM Renegade Mojave' 'KTM RC200' 'Bajaj Dominar 400'
 'Royal Enfield Classic 350' 'KTM RC390' 'Hyosung GT250R'
 'Royal Enfield Thunder 350' 'KTM 390 Duke ' 'Mahindra Mojo XT300'
 'Bajaj Pulsar RS200' 'Royal Enfield Bullet 350'
 'Royal Enfield Classic 500' 'Bajaj Avenger 220' 'Bajaj Avenger 150'
 'Honda CB Hornet 160R' 'Yamaha FZ S V 2.0' 'Yamaha FZ 16'
 'TVS Apache RTR 160' 'Bajaj Pulsar 150' 'Honda CBR 150' 'Hero Extreme'
 'Bajaj Avenger 220 dtsi' 'Bajaj Avenger 150 street' 'Yamaha FZ  v 2.0'
 'Bajaj Pulsar  NS 200' 'Bajaj Pulsar 220 F' 'TVS Apache RTR 180'
 'Hero Passion X pro' 'Bajaj Pulsar NS 200' 'Yamaha Fazer '
 'Honda Activa 4G' 'TVS Sport ' 'Honda Dream Yuga '
 'Bajaj Avenger Street 220' 'Hero Splend

In [85]:
X = data.drop(['Car_Name'], axis=1)
y = data['Selling_Price']

In [86]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [87]:
print('Columns with missing values in data: ', list(X_train.columns[X_train.isnull().any()]))

Columns with missing values in data:  []


In [88]:
from sklearn.preprocessing import OneHotEncoder

# This function returns the dataframe with encoded features
# @param dataframe - initial DataFrame
# @param features_names - names of features (columns) that need to be encoded
# @param encoder - encoder that should be used
# @return new_dataframe - new DataFrame with encoded features
def encode_features_one_hot(dataframe, features_names, encoder):
    new_features = encoder.transform(dataframe[features_names])
    new_columns_df = pd.DataFrame(new_features, columns = encoder.get_feature_names_out(features_names))
    dataframe.reset_index(drop=True, inplace=True)
    
    new_columns_df.reset_index(drop=True, inplace=True) 
    new_dataframe = pd.concat([dataframe, new_columns_df], axis=1)
    new_dataframe.drop(features_names, axis=1, inplace=True)
    return new_dataframe

# Encoding features
encoder = OneHotEncoder(sparse_output=False)

# Select columns that should be encoded (values are not numbers)
features_names = list(X.select_dtypes(exclude='number').columns)
print(features_names)

encoder.fit(X_train[features_names])
X_train = encode_features_one_hot(X_train, features_names, encoder)
X_test = encode_features_one_hot(X_test, features_names, encoder)
print(X_train.head())           

['Fuel_Type', 'Seller_Type', 'Transmission']
   Year  Selling_Price  Present_Price  Kms_Driven  Owner  Fuel_Type_CNG  \
0  2008           0.17           0.52      500000      0            0.0   
1  2014           7.20           9.90       48000      0            0.0   
2  2015           4.00           7.00       40026      0            0.0   
3  2013           6.95          18.61       40001      0            0.0   
4  2017           0.45           0.51        4000      0            0.0   

   Fuel_Type_Diesel  Fuel_Type_Petrol  Seller_Type_Dealer  \
0               0.0               1.0                 0.0   
1               1.0               0.0                 1.0   
2               0.0               1.0                 1.0   
3               0.0               1.0                 1.0   
4               0.0               1.0                 0.0   

   Seller_Type_Individual  Transmission_Automatic  Transmission_Manual  
0                     1.0                     1.0               

In [89]:
print('Columns with missing values in data: ', list(X_train.columns[X_train.isnull().any()]))
print('Columns with missing values in data: ', list(X_test.columns[X_train.isnull().any()]))


Columns with missing values in data:  []
Columns with missing values in data:  []


In [90]:
from sklearn.preprocessing import MinMaxScaler

# Scaling data
scaler = MinMaxScaler()
scaler.fit(X_train)

# Apply the scaler on both train and test features
X_train = pd.DataFrame(scaler.transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)
X_test

,Year,Selling_Price,Present_Price,Kms_Driven,Owner,Fuel_Type_CNG,Fuel_Type_Diesel,Fuel_Type_Petrol,Seller_Type_Dealer,Seller_Type_Individual,Transmission_Automatic,Transmission_Manual
0,0.866667,0.275076,0.369813,0.057504,0.000000,0.0,0.0,1.0,1.0,0.0,0.0,1.0
1,0.333333,0.004559,0.011974,0.051051,0.333333,0.0,0.0,1.0,0.0,1.0,0.0,1.0
2,0.800000,0.153495,0.149819,0.068663,0.000000,0.0,0.0,1.0,1.0,0.0,1.0,0.0
3,0.600000,0.088146,0.140072,0.106456,0.000000,0.0,0.0,1.0,1.0,0.0,0.0,1.0
4,0.666667,0.156535,0.266778,0.107592,0.000000,0.0,0.0,1.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
56,0.666667,0.005167,0.004177,0.041041,0.000000,0.0,0.0,1.0,0.0,1.0,0.0,1.0
57,0.800000,0.030699,0.032025,0.034034,0.000000,0.0,0.0,1.0,0.0,1.0,0.0,1.0
58,0.933333,0.011550,0.005291,0.007608,0.000000,0.0,0.0,1.0,0.0,1.0,1.0,0.0
59,0.733333,0.147416,0.199666,0.077077,0.000000,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [91]:
from sklearn.linear_model import LinearRegression

# Create a model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict the values    
y_pred = model.predict(X_test)  

# Calculate the error   
from sklearn.metrics import mean_squared_error, r2_score
error = mean_squared_error(y_test, y_pred) 
print('Mean squared error: ', error)
r2 = r2_score(y_test, y_pred)
print('R2 score: ', r2)

Mean squared error:  2.2548440490043326e-28
R2 score:  1.0
